In [ ]:
#SCRAPE

In [ ]:
!pip3 install -U nltk[twitter] 

     |████████████████████████████████| 1.4MB 4.3MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434678 sha256=48895bd8ae8abdfa4dec30db40435df555bc6e46ba5c1d271833eaf85f9e2e88
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import requests
import csv
import pandas as pd
from bs4 import BeautifulSoup
import os
from datetime import datetime, timedelta

import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
# Downloading today data from SGX using SGX API
today = datetime.now()
start_date = today-timedelta(1)
start_date = start_date.strftime('%Y%m%d')# Format must be in: '20190808'
end_date = today.strftime('%Y%m%d')
default_folder = './data'

# Start downloading data
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36"}
url = f'https://api.sgx.com/announcements/v1.0/?periodstart={start_date}_160000&periodend={end_date}_155959&pagestart=0&pagesize=20'
res = requests.get(url, headers)

# Store the downloaded data into dataframe (for easier access later)
json_data = res.json()['data']
df = pd.DataFrame(json_data)

# Assuming we are interested in the updates related to "LODHA DEVELOPERS INTERNATIONAL LIMITED"
stock_name = 'LODHA DEVELOPERS INTERNATIONAL LIMITED'

# Filtered out those unwanted company updates information
df = df[df['issuer_name']==stock_name]

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen
from urllib.request import Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'AMZN']

In [ ]:
finviz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finviz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass



Recent News Headlines for AAPL: 
Dow Jones Futures: President Trump Signs Stimulus Deal; Tesla On Cusp Of 'Historic Milestone' ( Dec-27-20 09:17PM )
12 Best Autonomous Vehicle Stocks to Buy for 2021 ( 06:51PM )
Dow Jones Futures Fall: As Stock Market Rally Pauses, Is Microsoft The Next Apple? Tesla On Cusp Of 'Historic Milestone' ( 06:19PM )


Recent News Headlines for TSLA: 
Is Tesla the next AOL? Thats what this strategist is warning ( Dec-27-20 07:24PM )
12 Best Autonomous Vehicle Stocks to Buy for 2021 ( 06:51PM )
CNBC's Michael Farr Believes Tesla Stock Is 'Stupidly Expensive' ( 05:49PM )


Recent News Headlines for AMZN: 
Barry Rosenstein and Jana Partners: Top 10 Stock Picks ( Dec-27-20 06:20PM )
A Data-Driven End to Capitalism as We Know It ( 06:00PM )
7 Bold Predictions for Tech Stocks in 2021 ( 08:45AM )


In [ ]:
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])

In [ ]:
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')

In [ ]:
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)



              Date     Time  neg    neu    pos  compound
Ticker                                                  
AAPL    2020-12-27  09:17PM  0.0  1.000  0.000    0.0000
AAPL    2020-12-27  06:51PM  0.0  0.656  0.344    0.6369
AAPL    2020-12-27  06:19PM  0.0  1.000  0.000    0.0000
AAPL    2020-12-27  06:00PM  0.0  1.000  0.000    0.0000
AAPL    2020-12-27  03:02PM  0.0  1.000  0.000    0.0000


              Date     Time    neg    neu    pos  compound
Ticker                                                    
TSLA    2020-12-27  07:24PM  0.194  0.806  0.000   -0.3400
TSLA    2020-12-27  06:51PM  0.000  0.656  0.344    0.6369
TSLA    2020-12-27  05:49PM  0.273  0.727  0.000   -0.4588
TSLA    2020-12-27  11:38AM  0.260  0.740  0.000   -0.3535
TSLA    2020-12-27  09:09AM  0.171  0.650  0.179    0.0258


              Date     Time  neg    neu    pos  compound
Ticker                                                  
AMZN    2020-12-27  06:20PM  0.0  0.816  0.184    0.2023
AMZN    202

In [ ]:
df

,Mean Sentiment
Ticker,
AMZN,0.16
AAPL,0.08
TSLA,0.03


In [ ]:
dataframe

,Date,Time,neg,neu,pos,compound
Ticker,,,,,,
AMZN,2020-12-27,06:20PM,0.000,0.816,0.184,0.2023
AMZN,2020-12-27,06:00PM,0.000,1.000,0.000,0.0000
AMZN,2020-12-27,08:45AM,0.000,0.698,0.302,0.3818
AMZN,2020-12-27,07:12AM,0.000,1.000,0.000,0.0000
AMZN,2020-12-27,06:25AM,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...
AMZN,2020-12-22,03:53AM,0.000,1.000,0.000,0.0000
AMZN,2020-12-21,07:00PM,0.000,0.635,0.365,0.5574
AMZN,2020-12-21,05:29PM,0.000,1.000,0.000,0.0000


In [ ]:
# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'AMZN']

# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])
        
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')


# View Data 
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)



Recent News Headlines for AAPL: 
Dow Jones Futures: President Trump Signs Stimulus Deal; Tesla On Cusp Of 'Historic Milestone' ( Dec-27-20 09:27PM )
12 Best Autonomous Vehicle Stocks to Buy for 2021 ( 06:51PM )
Dow Jones Futures Fall: As Stock Market Rally Pauses, Is Microsoft The Next Apple? Tesla On Cusp Of 'Historic Milestone' ( 06:19PM )


Recent News Headlines for TSLA: 
Is Tesla the next AOL? Thats what this strategist is warning ( Dec-27-20 07:24PM )
12 Best Autonomous Vehicle Stocks to Buy for 2021 ( 06:51PM )
CNBC's Michael Farr Believes Tesla Stock Is 'Stupidly Expensive' ( 05:49PM )


Recent News Headlines for AMZN: 
Barry Rosenstein and Jana Partners: Top 10 Stock Picks ( Dec-27-20 06:20PM )
A Data-Driven End to Capitalism as We Know It ( 06:00PM )
7 Bold Predictions for Tech Stocks in 2021 ( 08:45AM )


              Date     Time  neg    neu    pos  compound
Ticker                                                  
AAPL    2020-12-27  09:27PM  0.0  1.000  0.000    0.0000